In [1]:
import numpy as np
import wikipediaapi

In [2]:
class sommet():
    def __init__(self,label,value=0):
        self.label = label
        self.value = value

class arete():
    def __init__(self,sommet1,sommet2):
        self.sommet1 = sommet1
        self.sommet2 = sommet2

class graphe():
    def __init__(self,sommet,liste_sommet_adjacent =()):
        self.nb_arete = 0
        self.nb_sommet = 1
        # orienté
        self.dict_successeur = {sommet:liste_sommet_adjacent}

    def ajoute_sommet(self,sommet,liste_sommet_adjacent):
        if sommet not in self.dict_adjacent:
            self.dict_adjacent[sommet] = liste_sommet_adjacent
        else:
            for sommet_adjacent in liste_sommet_adjacent:
                if sommet_adjacent not in self.dict_adjacent:
                    self.dict_adjacent[sommet] += (sommet_adjacent)

    def ajoute_arete(self,liste_arete):
        for arete in liste_arete:
            if arete.sommet1 in self.dict_adjacent:
                if arete.sommet2 not in self.dict_adjacent[arete.sommet1]:
                    self.dict_adjacent[arete.sommet1] += (arete.sommet2)

"""
wiki_wiki = wikipediaapi.Wikipedia('en')

page_py = wiki_wiki.page('Python_(programming_language)')
print("Page - Exists: %s" % page_py.exists())
# Page - Exists: True

page_missing = wiki_wiki.page('NonExistingPageWithStrangeName')
print("Page - Exists: %s" %     page_missing.exists())
# Page - Exists: False """

'\nwiki_wiki = wikipediaapi.Wikipedia(\'en\')\n\npage_py = wiki_wiki.page(\'Python_(programming_language)\')\nprint("Page - Exists: %s" % page_py.exists())\n# Page - Exists: True\n\npage_missing = wiki_wiki.page(\'NonExistingPageWithStrangeName\')\nprint("Page - Exists: %s" %     page_missing.exists())\n# Page - Exists: False '

In [5]:
def power_iteration(dict1,tab,N,alpha):
    page_rank_final=np.zeros(N)
    page_rank=tab
    constante=(1-alpha)/N

    for cles in dict1:
        print("cles",cles)
        somme=0
        for cles2 in dict1:
            if cles in dict1[cles2]:
                print("sommet",cles2)
                somme=somme+(tab[cles2-1]/len(dict1[cles2]))
                print("valeur du page rank",tab[cles2-1])
                print("nombre d'arcs sortants",len(dict1[cles2]))
        page_rank_final[cles-1]=alpha*somme+constante
    return page_rank_final

    #Test:
N2=3
dict2={1:{2,3}, 2:{3}, 3:{1}}
tab2=np.ones(N2)/N2
for i in range(10):
    tab2=power_iteration(dict2,tab2,N2,0.85)
    print(tab2)


cles 1
sommet 3
valeur du page rank 0.3333333333333333
nombre d'arcs sortants 1
cles 2
sommet 1
valeur du page rank 0.3333333333333333
nombre d'arcs sortants 2
cles 3
sommet 1
valeur du page rank 0.3333333333333333
nombre d'arcs sortants 2
sommet 2
valeur du page rank 0.3333333333333333
nombre d'arcs sortants 1
[0.33333333 0.19166667 0.475     ]
cles 1
sommet 3
valeur du page rank 0.475
nombre d'arcs sortants 1
cles 2
sommet 1
valeur du page rank 0.3333333333333333
nombre d'arcs sortants 2
cles 3
sommet 1
valeur du page rank 0.3333333333333333
nombre d'arcs sortants 2
sommet 2
valeur du page rank 0.19166666666666668
nombre d'arcs sortants 1
[0.45375    0.19166667 0.35458333]
cles 1
sommet 3
valeur du page rank 0.3545833333333333
nombre d'arcs sortants 1
cles 2
sommet 1
valeur du page rank 0.45375
nombre d'arcs sortants 2
cles 3
sommet 1
valeur du page rank 0.45375
nombre d'arcs sortants 2
sommet 2
valeur du page rank 0.19166666666666668
nombre d'arcs sortants 1
[0.35139583 0.24284375 0